# OPTIMIZANDO CODIGOS PARA ALTA PERFORMANCE

O pacote `BenchmarkTools` facilita o acompanhamento de desempenho do código em Julia, fornecendo uma estrutura para escrever e executar grupos de benchmarks, bem como comparar resultados. `BenchmarkTools` foi criado para facilitar as seguintes tarefas:

+ Organizar coleções de benchmarks em conjuntos de benchmarks gerenciáveis
+ Configurar, salvar e recarregar parâmetros de referência para conveniência, precisão e consistência
+ Executar benchmarks de forma a obter previsões de desempenho razoáveis e consistentes
+ Analisar e comparar resultados para determinar se uma alteração de código causou prejuízos ou melhorias

Sintaxe Básica:
```julia
@benchmark comando/função

Out:
  BenchmarkTools.Trial: 
      samples:          1270
      evals/sample:     1
      time tolerance:   5.00%
      memory tolerance: 1.00%
      memory estimate:  7.63 mb
      allocs estimate:  3
      minimum time:     3.03 ms (0.00% GC)
      median time:      3.80 ms (0.00% GC)
      mean time:        3.93 ms (3.97% GC)
      maximum time:     6.94 ms (6.20% GC)
  
```

In [9]:
Pkg.add("BenchmarkTools")

INFO: Cloning cache of BenchmarkTools from https://github.com/JuliaCI/BenchmarkTools.jl.git
INFO: Installing BenchmarkTools v0.0.5
INFO: Package database updated


In [9]:
using BenchmarkTools

## TIPOS DEFINIDOS EM FUNÇÕES

Julia é uma linguagem dinamicamente tipada que ao contrário Java ou C, o programador não precisa especificar o tipo fixo de cada variável no código. Os tipos são muito importantes em Julia, de tal forma que o melhor desempenho ocorre quando são utilizadas informações de tipo para todos os dados no código. Um tipo mal definido pode atrasar e muito o desempenho de uma função/codigo. A melhor forma de obter um bom desempenho é casar o tipo do dado a ser trabalhado com o tipo que a função vai calcular.

No exemplo abaixo, temos uma função que soma elementos de um vetor. No primeiro teste a função `soma_vetor1` recebe um vetor de qualquer tipo e trabalha com "soma" do tipo `Float (soma = 0.0)`. No segundo teste a mesma função e vetores, porém a variável soma é do tipo inteiro.

In [10]:
# o comando i = x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetor1(x)
    soma = 0.0
    for i = x
        soma = soma + i
    end
    return soma
end

soma_vetor1 (generic function with 1 method)

In [6]:
x1 = collect(0:0.0001:5000);        # Array{Float64,1},
x2 = linspace(0,5000,50000001);     # LinSpace{Float64}
x3 = 0:0.0001:5000;                 # FloatRange{Float64}

length(x1) , length(x2) , length(x3)

(50000001,50000001,50000001)

In [7]:
# confirmando o tipo
typeof(x1) , typeof(x2) , typeof(x3)

(Array{Float64,1},LinSpace{Float64},FloatRange{Float64})

In [11]:
@benchmark soma_vetor1(x1)

BenchmarkTools.Trial: 
  samples:          75
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  16.00 bytes
  allocs estimate:  1
  minimum time:     62.97 ms (0.00% GC)
  median time:      68.45 ms (0.00% GC)
  mean time:        67.27 ms (0.00% GC)
  maximum time:     69.85 ms (0.00% GC)

In [12]:
@benchmark soma_vetor1(x2)

BenchmarkTools.Trial: 
  samples:          18
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  16.00 bytes
  allocs estimate:  1
  minimum time:     280.91 ms (0.00% GC)
  median time:      281.45 ms (0.00% GC)
  mean time:        281.57 ms (0.00% GC)
  maximum time:     282.57 ms (0.00% GC)

In [13]:
@benchmark soma_vetor1(x3)

BenchmarkTools.Trial: 
  samples:          18
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  16.00 bytes
  allocs estimate:  1
  minimum time:     281.14 ms (0.00% GC)
  median time:      281.55 ms (0.00% GC)
  mean time:        281.81 ms (0.00% GC)
  maximum time:     285.54 ms (0.00% GC)

O tipo Array{Float64,1} definido pelo ` collect` apresentou melhor desempenho de tempo.

A função agora modificada. Note que "soma" agora é do tipo inteiro, mas está recebendo um vetor do tipo Float

In [16]:
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetor2(x)
    soma = 0             # so esse datalhe ja é suficiente para reduzir o desempenho
    for i in x
        soma = soma + i
    end
    return soma
end

soma_vetor2 (generic function with 1 method)

In [17]:
@benchmark soma_vetor2(x1)

BenchmarkTools.Trial: 
  samples:          4
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  2.24 gb
  allocs estimate:  150000003
  minimum time:     1.12 s (10.08% GC)
  median time:      1.27 s (9.85% GC)
  mean time:        1.28 s (9.82% GC)
  maximum time:     1.45 s (9.25% GC)

In [18]:
@benchmark soma_vetor2(x2)

BenchmarkTools.Trial: 
  samples:          5
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  2.24 gb
  allocs estimate:  150000003
  minimum time:     1.22 s (9.37% GC)
  median time:      1.22 s (9.41% GC)
  mean time:        1.23 s (9.34% GC)
  maximum time:     1.26 s (9.19% GC)

In [19]:
@benchmark soma_vetor2(x3)

BenchmarkTools.Trial: 
  samples:          4
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  2.24 gb
  allocs estimate:  150000003
  minimum time:     1.30 s (9.41% GC)
  median time:      1.50 s (9.43% GC)
  mean time:        1.49 s (9.40% GC)
  maximum time:     1.68 s (9.01% GC)

Veja que todos os testes apresentaram resultados muito ruins em função do tipo de "soma" ser inteiro.

## NÃO USE VARIÁVEIS GLOBAIS EM FUNÇÕES

Variáveis globais são uma tentação na vida de qualquer programador. Porém...

In [6]:
using BenchmarkTools

In [3]:
# função calcula a soma a partir de uma variável global X
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetorG()
    soma = 0.0             # cuidado! soma é tipo float, se não piora mais ainda
    for i in x
        soma = soma +  i
    end
    return soma
end

soma_vetorG (generic function with 1 method)

In [4]:
x = collect(0:0.0001:5000);

In [7]:
@benchmark soma_vetorG()

BenchmarkTools.Trial: 
  samples:          1
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  3.73 gb
  allocs estimate:  199999494
  minimum time:     5.53 s (5.02% GC)
  median time:      5.53 s (5.02% GC)
  mean time:        5.53 s (5.02% GC)
  maximum time:     5.53 s (5.02% GC)

A mesma função só que usando variável local passada para função

In [8]:
# função calcula a soma a partir de uma variável local X passada como valor
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetorL(x)
    soma = 0.0             # cuidado! soma é tipo float
    for i in x
        soma = soma + i
    end
    return soma
end

soma_vetorL (generic function with 1 method)

In [9]:
x = collect(0:0.0001:5000);

In [10]:
@benchmark soma_vetorL(x)

BenchmarkTools.Trial: 
  samples:          90
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  16.00 bytes
  allocs estimate:  1
  minimum time:     52.55 ms (0.00% GC)
  median time:      55.81 ms (0.00% GC)
  mean time:        55.84 ms (0.00% GC)
  maximum time:     66.38 ms (0.00% GC)

Diferença absurda de tempo de alocação de memória entre o primeiro caso e segundo caso. 

## UTILIZE FUNÇÕES PARA SUBSTITUIR BLOCOS DE CÓDIGO

In [25]:
Pkg.add("BenchmarkTools")

INFO: Nothing to be done


In [26]:
using BenchmarkTools

Bloco de código que utiliza variáveis globais para calcular a soma todos os elementos de um vetor

In [20]:
tic()
x = collect(0:0.0001:5000)

soma = 0.0             # cuidado! soma é tipo float
for i in x1
    soma = soma + i
end

display(soma)

toc()

1.2500000249999998e11

elapsed time: 6.405834042 seconds


6.405834042

Função que substitui o procedimento acima e usa variáveis locais

In [15]:
# função calcula a soma a partir de uma variável local X passada como valor
# o comando i in x faz a variável i percorrer todo o vetor x
# pode ser utilizado for i = 1:length(x)

function soma_vetorL(x)
    soma = 0.0             # cuidado! soma é tipo float
    for i in x
        soma = soma + i
    end
    return soma
end

soma_vetorL (generic function with 1 method)

In [22]:
x = collect(0:0.0001:5000);

In [23]:
@benchmark soma_vetorL(x)

BenchmarkTools.Trial: 
  memory estimate:  16.00 bytes
  allocs estimate:  1
  --------------
  minimum time:     54.147 ms (0.00% GC)
  median time:      55.326 ms (0.00% GC)
  mean time:        55.792 ms (0.00% GC)
  maximum time:     63.881 ms (0.00% GC)
  --------------
  samples:          90
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

Veja que a diferença de tempo é absurda!

## CRIAÇÃO DE MATRIZES

Quando for criar uma matriz a partir de uma função, use `List Compreehsion` ao invés de laços `FOR`, além de deixar mais claro o código é bem mais rápido.

In [2]:
using BenchmarkTools

In [41]:
# função for
function matriz_for(n)
    matriz  = ones(n,n)
    for i = 1:n
        for j = 1:n
            matriz[i,j] = 0.5*i^2 + 0.3j^2
        end
    end
    return matriz
end

matriz_for (generic function with 1 method)

In [42]:
@benchmark matriz_for(20000)

BenchmarkTools.Trial: 
  samples:          2
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  2.98 gb
  allocs estimate:  2
  minimum time:     4.27 s (0.25% GC)
  median time:      4.30 s (0.87% GC)
  mean time:        4.30 s (0.87% GC)
  maximum time:     4.33 s (1.47% GC)

In [43]:
# use o garbage collector para remover o lixo da memória
gc()
gc()

In [5]:
# função LC
function matriz_lc(n)
   return [0.5*i^2 + 0.3*j^2 for i = 1:n, j = 1:n]
end

matriz_lc (generic function with 1 method)

In [14]:
@benchmark matriz_lc(20000)

BenchmarkTools.Trial: 
  samples:          3
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  2.98 gb
  allocs estimate:  3
  minimum time:     1.90 s (0.56% GC)
  median time:      1.95 s (2.63% GC)
  mean time:        1.93 s (2.12% GC)
  maximum time:     1.95 s (2.62% GC)

In [ ]:
# use o garbage collector para remover o lixo da memória
gc()
gc()

## QUAL A MELHOR: FUNÇÕES GENÉRICAS, SIMBÓLICAS OU ANÔNIMAS?

Uma dúvida que ocorre devido as possibilidades de escolhas é qual função apresenta melhor desenpenho de cálculo. As funções anônimas foram otimizadas na versão 0.5 e não há necessidade do pacote `FastAnonymous` como era nas versões anteriores.

* **Funções Genéricas**

|Vantagens   |Desvantagens |
|------------|-------------|
|Similares às funções tipo `Fuction`|Só permitem cálculo de funções definidas|
|Fáceis de escrever e bem próximas da escrita de uma função da matemática|A redefinição|

* **Funções Simbólicas**

|Vantagens   |Desvantagens |
|------------|-------------|
|Permitem o cálculo simbólico entre variáveis|Muito lentas em relação as funções genéricas e anônimas        |
|Podem ser utilizadas como genéricas e variável simbólica|afdasdfa                                                |

* **Funções Anônimas**

|Vantagens   |Desvantagens |
|------------|-------------|
|Velocidade um pouco melhor que as funções genéricas|afdasdfa|
|asdfasdf|afdasdfa|

In [5]:
using BenchmarkTools

In [1]:
# Genérica
f_gen(x , y) = x^2 + y^3 + x^5 + y^7 + x^9 + y^11

f_gen (generic function with 1 method)

In [2]:
using SymPy
@syms x y

(x,y)

In [11]:
# Simbólica
f_simb = x^2 + y^3 + x^5 + y^7 + x^9 + y^11

 9    5    2    11    7    3
x  + x  + x  + y   + y  + y 

In [24]:
x_vetor = rand(200, 200)
y_vetor = rand(200, 200);

In [25]:
# Função Genérica
@benchmark f_gen.(x_vetor,y_vetor)

BenchmarkTools.Trial: 
  memory estimate:  313.94 kb
  allocs estimate:  38
  --------------
  minimum time:     172.648 μs (0.00% GC)
  median time:      179.037 μs (0.00% GC)
  mean time:        193.842 μs (4.03% GC)
  maximum time:     1.013 ms (69.42% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [26]:
# Função Simbólica
@benchmark f_simb.(x_vetor,y_vetor)

BenchmarkTools.Trial: 
  memory estimate:  334.78 mb
  allocs estimate:  9720068
  --------------
  minimum time:     91.443 s (0.13% GC)
  median time:      91.443 s (0.13% GC)
  mean time:        91.443 s (0.13% GC)
  maximum time:     91.443 s (0.13% GC)
  --------------
  samples:          1
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [27]:
# Função Anônima
@benchmark ((x,y) -> x^2 + y^3 + x^5 + y^7 + x^9 + y^11).(x_vetor,y_vetor)

BenchmarkTools.Trial: 
  memory estimate:  313.94 kb
  allocs estimate:  38
  --------------
  minimum time:     173.810 μs (0.00% GC)
  median time:      177.172 μs (0.00% GC)
  mean time:        191.173 μs (4.18% GC)
  maximum time:     1.098 ms (66.09% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

## OTIMIZANDO UBUNTU E DERIVADOS

O Ubuntu e outros sistemas linux, possuem uma configuração de **"Concurrency"** que permite habilitar todos os núcleos, isso faz com que o Boot do seu sistema seja mais rápido que o normal e pode melhorar o desempenho do processamento numérico da Julia. Para habilitar o recurso, digite no shell:

```bash
sudo gedit /etc/init.d/rc
```
Para evitar problemas, salve o arquivo como **rc.back**, caso ocorra algum erro ou instabilidade do sistema. Procure as linhas **CONCURRENCY=makefile** e descomente, e comente a linha **#CONCURRENCY=none**. Veja o código abaixo:

    # Check if we are able to use make like booting.  It require the
    # insserv package to be enabled. Boot concurrency also requires
    # startpar to be installed.
    #
    CONCURRENCY=makefile # descomente a linha
    # disable startpar, incompatible with "task" upstart jobs
    #CONCURRENCY=none   # comente a linha
    
Reinicie e veja a diferença :)

## PARALELISMO 

### TRABALHAR COM CORES
Para adicionar cores utilize:
```julia
addproces(n)
```
Sendo **n** o número de cores que deseja criar as instâncias. Uma boa dica é usar ``addroces()`` dessa forma Julia cria automaticamente o número de cores que melhor se ajusta à máquina.

In [39]:
# visualizar o numero de cores do processador físico
Sys.CPU_CORES

4

In [1]:
# verificar o número de cores disponíveis e ID
procs()

1-element Array{Int64,1}:
 1

In [4]:
# adicionar o numero de cores do computador
addprocs()

4-element Array{Int64,1}:
 2
 3
 4
 5

In [2]:
# remover cores rmprocs(ID_CORE)
# para remover mais de um core use rmprocess([vetor_IDCORE])
# Para remover todos os alocados use rmprocs(procs()) 
rmprocs(procs())

:ok

In [3]:
procs()

1-element Array{Int64,1}:
 1

### PARALLEL FOR E  PARALLEL MAP

Um loop for com um grande número de iterações e compreensões de lista, são melhores desempenhados com execução paralela através  da macro `@parallel`. Sintaxe:
```julia
@parallel (operador) for var = range
    instruções
end
```

In [28]:
using BenchmarkTools

**Função Somatório **

In [65]:
# Função Soma.
# Laço for de uma soma de 1 a 1000000000
# de números aleatórios

function serial_add(n)
    s = 0.0
    for i = 1:n
        s = s + randn()
    end
    return s
end 

serial_add (generic function with 1 method)

In [67]:
@benchmark serial_add(1000000000)

BenchmarkTools.Trial: 
  memory estimate:  0.00 bytes
  allocs estimate:  0
  --------------
  minimum time:     6.412 s (0.00% GC)
  median time:      6.412 s (0.00% GC)
  mean time:        6.412 s (0.00% GC)
  maximum time:     6.412 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [33]:
# verificar o número de processadores
procs()

1-element Array{Int64,1}:
 1

In [34]:
addprocs()

4-element Array{Int64,1}:
 2
 3
 4
 5

In [68]:
# função paralela
function parallel_add(n)
    
  @parallel (+) for i = 1:n
        randn()                  
    end
    
end

parallel_add (generic function with 1 method)

In [70]:
# execute mais de uma vez
@benchmark parallel_add(1000000000)

BenchmarkTools.Trial: 
  memory estimate:  50.44 kb
  allocs estimate:  609
  --------------
  minimum time:     2.623 s (0.00% GC)
  median time:      2.627 s (0.00% GC)
  mean time:        2.627 s (0.00% GC)
  maximum time:     2.631 s (0.00% GC)
  --------------
  samples:          2
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

**Função alocação de dados em uma Matriz**

In [90]:
# Função paralela

function alo_matriz1(n)
    matriz = zeros(n,n)
    @parallel for i = 1:n
        for j = 1:n
            matriz[i,j] = rand()*0.25
        end
    end
    #return matriz
end

alo_matriz1 (generic function with 1 method)

In [97]:
@benchmark alo_matriz1(10000)

LoadError: LoadError: OutOfMemoryError()
while loading In[97], in expression starting on line 183

In [96]:
# use o garbage collector para remover o lixo da memória
gc()
gc()

In [25]:
# Função normal
function alo_matriz2(n)
    matriz = zeros(n,n)
    for i = 1:n
        for j = 1:n
            matriz[i,j] = rand()*0.25
        end
    end
    return matriz
end

alo_matriz2 (generic function with 1 method)

In [26]:
@benchmark alo_matriz2(10000)

BenchmarkTools.Trial: 
  samples:          3
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  762.94 mb
  allocs estimate:  2
  minimum time:     1.72 s (4.08% GC)
  median time:      1.77 s (3.94% GC)
  mean time:        1.76 s (3.85% GC)
  maximum time:     1.77 s (3.54% GC)

In [31]:
# use o garbage collector para remover o lixo da memória
gc()
gc()

### ParallelAccelerator

ParallelAccelerator é um pacote para acelerar programas de Julia voltados principalmente para o cálculo de matrizes e vetores. Com a macro `@acc` **ParallelAccelerator** compila partes do programa e elimina automaticamente funções desnecessárias e também paraleliza e vetoriza muitas operações de dados em paralelo. Além de acelerar o cálculo, `@acc` optimiza o consumo de memória, logo é muito útil para grandes matrizes. 

ParallelAccelerator é parte do projeto de script de alto desempenho (HPS) da Intel Labs. 



In [1]:
Pkg.add("ParallelAccelerator")

INFO: Nothing to be done


In [24]:
using BenchmarkTools

In [25]:
NP(n) = [i^2 + j^3 + i^5 + j^7 + i^9 + j^11 for i = 1:n, j = 1:n];

In [26]:
@benchmark NP(10000)

BenchmarkTools.Trial: 
  samples:          2
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  762.94 mb
  allocs estimate:  3
  minimum time:     3.48 s (0.46% GC)
  median time:      3.53 s (1.27% GC)
  mean time:        3.53 s (1.27% GC)
  maximum time:     3.58 s (2.06% GC)

In [99]:
# use o garbage collector para remover o lixo da memória
gc()
gc()

In [100]:
# verificar os cores alocados. Se houver mais de um até o número de cores físicos OK.
procs()

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

In [14]:
# se houver apenas um cores execute
addprocs()

4-element Array{Int64,1}:
 6
 7
 8
 9

In [104]:
Pkg.add("ParallelAccelerator")

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of ParallelAccelerator
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [105]:
using ParallelAccelerator

INFO: Precompiling module DataStructures.


In [108]:
@acc PA(n) = [i.^2 + j.^3 + i.^5 + j.^7 + i.^9 + j^.11 for i = 1:n, j = 1:n];

In [111]:
# execute sempre mais de uma vez para obter o melhor resultado
@timev PA(10000);

  4.515833 seconds (35 allocations: 762.941 MB, 2.22% gc time)
elapsed time (ns): 4515833370
gc time (ns):      100031677
bytes allocated:   800001968
pool allocs:       35
GC pauses:         1


In [114]:
# use o garbage collector para remover o lixo da memória
gc()
gc()

### DistributedArrays

In [ ]:
Pkg.add("DistributedArrays")

In [34]:
using BenchmarkTools

In [33]:
function array(n)
    return ([i^2 + j^3 + i^5 + j^7 + i^9 + j^11 for i = 1:n, j = 1:n]);
end

array (generic function with 1 method)

In [40]:
@benchmark array(7000)

BenchmarkTools.Trial: 
  samples:          3
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%
  memory estimate:  373.84 mb
  allocs estimate:  3
  minimum time:     1.70 s (0.67% GC)
  median time:      1.77 s (3.56% GC)
  mean time:        1.76 s (2.62% GC)
  maximum time:     1.82 s (3.52% GC)

In [1]:
# use o garbage collector para remover o lixo da memória
gc()
gc()

![Execução Sem Paralelismo](Arrays-sem-paralelismo.png)

Antes de executar o código é necesário verificar se há outros cores disponíveis. Caso exista somente 1, é necessário adicionar novos cores e vincular o `DistributedArrays` ao macro `@everywhere` seguindo a ordem:

    1º adicione os cores addprocs(n*)
    2º use @everywhere using DistributedArrays
    3º execute o código com o macro @Darray 
**n** corresponde ao número de cores. Utilizando `addprocs()`(o mesmo que usar ``addprocs(Sys.CPU_CORES)``) Julia cria automaticamente o número de cores que melhor se ajustam à máquina. Sempre execute uma segunda vez o código, a primeira tem péssimo desempenho.

Caso execute novamente `addprocs(n)` e depois `@Darray`, os cores novos serão eliminados da lista de processos e será utilizado somente os cores vinculados ao macro `@everywhere`. 

In [13]:
# visualizar o numero de cores do processador físico
Sys.CPU_CORES

4

In [2]:
# verificar o número de cores disponíveis e ID
procs()

1-element Array{Int64,1}:
 1

In [3]:
# adiciona automaticanente a melhor quantidade de acordo com a quantidade de cores disponíveisa
addprocs()

4-element Array{Int64,1}:
 2
 3
 4
 5

In [4]:
# vincular aos cores
@everywhere using DistributedArrays

In [5]:
function Darray(n)
    return @DArray ([i^2 + j^3 + i^5 + j^7 + i^9 + j^11 for i = 1:n, j = 1:n]);
end

Darray (generic function with 1 method)

In [8]:
# a primeira execução é ruim. Execute uma segunda vez após gc()
# Aqui o @benchmark não é adequado.
@timev  Darray(7000)

# use o garbage collector para remover o lixo da memória
gc()
gc()

  0.850469 seconds (1.05 k allocations: 74.328 KB)
elapsed time (ns): 850468583
bytes allocated:   76112
pool allocs:       1041
non-pool GC allocs:6


![](Arrays-com-paralelismo.png)

In [30]:
#remover os nucleos alocados 
rmprocs(procs())

:ok